In [ ]:
import csv
from pickle import load
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import load_model
from keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

In [ ]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [ ]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [ ]:
# load text
input_path = "D:\\AML-Project\\Raw data\\"
output_path = "D:\AML-Project\Data\\"
in_filename = 'elonmusk_tweets.csv'

In [ ]:
sentences = []
with open(input_path + in_filename) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count+=1
        else:
            low = row[2][2:-1].lower()
            sentences.append(low)
            line_count+=1
    print(f'Processed {line_count} lines.')
    
raw_text = " ".join(sentences)
print(sentences)

In [ ]:
# clean
count = 0
for tweet in sentences:
    temp = tweet.split()
    for word in temp:
        if '/' in word:
            temp.remove(word)
        elif '\\' in word:
            temp.remove(word)
        elif 'rt' in word:
            temp.remove(word)
        elif '@' in word:
            temp.remove(word)
    sentences[count] = temp
    count += 1
sentences

[]

In [ ]:
# organize into sequences of characters
sequences = list()
for tweet in sentences:
    for i in range(len(tweet)-4):
        seq = tweet[i]+" "+tweet[i+1]+" "+tweet[i+2]+" "+tweet[i+3]+" "+tweet[i+4]
        sequences.append(seq)
print('Total Sequences: %d' % len(sequences))
sequences

Total Sequences: 0


[]

In [ ]:
# save sequences to file
filename = 'elon_sequences_WL1.txt'
out_filename = output_path+filename
save_doc(sequences, out_filename)
sequences

In [ ]:
input_path = 'C:\\Users\\iD Student\\Documents\\PPNN\\Processed Data\\'
# load the model
model = load_model(input_path+'model_WL1.h5')

In [ ]:
# load the mapping
mapping = load(open(input_path+'mapping_WL1.pkl', 'rb'))

In [ ]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[word] for word in in_text.split()]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# one hot encode
		encoded = to_categorical(encoded, num_classes=len(mapping))
		encoded = encoded.reshape(1, encoded.shape[1], encoded.shape[2])
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out = ''
		for word, index in mapping.items():
			if index == yhat:
				out = word
				break
		# append to input
		in_text = in_text+ " " + out
	return in_text

In [ ]:
# test
print(generate_seq(model, mapping, 2, 'got mars', 20))
# test
print(generate_seq(model, mapping, 2, 'spend some', 20))
# test
print(generate_seq(model, mapping, 2, 'solar power', 20))

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [ ]:
# load
input_path = "D:\\Github\\PPPNN\\Processed Data\\"
in_filename = 'elon_sequences_WL1.txt'
raw_text = load_doc(input_path+in_filename)
lines = raw_text.split('\n')

In [ ]:
words = []
for line in lines:
	temp_words = line.split()
	for word in temp_words:
		if word not in words:
			words.append(word)

mapping = dict((c, i) for i, c in enumerate(words))
print(mapping)

In [ ]:
sequences = list()
for line in lines:
	# integer encode line
	encoded_seq = [mapping[word] for word in line.split()]
	# store
	sequences.append(encoded_seq)
print(sequences)

In [ ]:
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

In [ ]:
sequences = array(sequences)
print(sequences)
X, y = sequences[:,:-3], sequences[:,-3]

In [ ]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
# define model
model = Sequential()
model.add(LSTM(1000, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

In [ ]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=2)

In [ ]:
# save the model to file
model.save(input_path+'model_WL1.h5')
# save the mapping
dump(mapping, open(input_path+'mapping_WL1.pkl', 'wb'))

In [ ]:
print("aaa")